In [1]:
import sys
sys.path.insert(0, '..')
from app.SIWeaviateClient import SIWeaviateClient
from app.schemas import Document, ImageChunk, ImageMetadata, TextChunk, TextMetadata
from app.processing.image.SIImageDescription import SIImageDescription
from app.processing.text.SIITranslator import SITranslator
from app.processing.BaseDocumentProcessor import BaseDocumentProcessor
import os
import fitz
import io
from PIL import Image
from unstructured.partition.pdf import partition_pdf
import re

image_descriptor = SIImageDescription()
translator = SITranslator()



/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:45: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: DeprecationWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", DeprecationWarning)


In [2]:
from app.processing.PDFProcessor import PDFProcessor
import os
import glob

def get_all_pdfs():
    pdf_files = []
    for root, dirs, files in os.walk("./data"):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files


with SIWeaviateClient() as client:
    pdf_paths = get_all_pdfs()
    for pdf_path in pdf_paths:
        print("==========================================")
        print(pdf_path)
        pdf = PDFProcessor(client, image_descriptor, translator, pdf_path)



./data/UniversScience/dossiers-pédagogiques/marc/cadran_solaire.pdf
The image is a logo for a company called La Mediation Scientificique. The logo is written in black text on a white background. The company name is in a bold font and is centered in the middle of the image. 
L'image est un logo pour une entreprise appelée La Médiation Scientifique. Le logo est écrit en texte noir sur un fond blanc. Le nom de l'entreprise est dans une police en gras et est centré au milieu de l'image.
A black and white logo with the word "cite" written in capital letters. Below the word, "sciences etindustrie" is written in smaller letters. The background of the image is plain white.
Un logo noir et blanc avec le mot «  cite  » écrit en lettres majuscules. Au-dessous du mot, «  science et industrie  » est écrit en lettres plus petites. Le fond de l'image est blanc clair.
A hand is holding a small globe. The globe is blue and white. There is a clock on top of the globe. 
Une main tient un petit globe. Le 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


The image is a logo for a company called La Mediation Scientificique. The logo is written in black text on a white background. The company name is in a bold font and is centered in the middle of the image. 
L'image est un logo pour une entreprise appelée La Médiation Scientifique. Le logo est écrit en texte noir sur un fond blanc. Le nom de l'entreprise est dans une police en gras et est centré au milieu de l'image.
A man is wearing a silver suit and a white helmet. He is holding a white sign in his hand. The sign has the word Mars written on it in black letters. The man is giving a thumbs up. 
Un homme porte une combinaison d'argent et un casque blanc. Il tient un signe blanc dans sa main. Le signe a le mot Mars écrit dessus en lettres noires. L'homme donne un pouce vers le haut.
A woman and a child are sitting on a bench in front of a large screen. The woman is wearing a pink jacket and carrying a black purse. The child is wearing an orange jacket and blue jeans. There is a white sig

Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


The image is a black square with white words on it. The words are in French and are arranged in a grid-like pattern. Each word is written in a different language. The first word is "education" and the second is "conférences". The third is "évenements". The fourth is "animations". The fifth is "formation".
L'image est un carré noir avec des mots blancs sur lui. Les mots sont en français et sont arrangés dans un motif ressemblant à une grille. Chaque mot est écrit dans une langue différente. Le premier mot est « éducation » et le second est « conférences ». Le troisième est « événements ». Le quatrième est « animations ». Le cinquième est « formation ».
The word "university" is written in red capital letters. The letters are all the same size and shape. The background of the image is white.
Le mot «  université  » est écrit en lettres majuscules rouges. Les lettres ont toutes la même taille et la même forme. Le fond de l'image est blanc.
A satellite is shown in the sky. The satellite is 

/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/pdfminer/converter.py:238: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmp_ys9hzb_'>
  item = LTChar(
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/pdfminer/converter.py:238: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmpmzkp54yv'>
  item = LTChar(
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/pdfminer/converter.py:238: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmph10p2pmf'>
  item = LTChar(
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/typing.py:274: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmppgm1pa92'>
  return cached(*args, **kwds)
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/typing.py:274: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmpb76hwhen'>
  return cached(*args, **kwds)
/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/typing.py:274: ResourceW

In [ ]:
combined = images + texts

# Sort the combined list
sorted_list = sorted(combined, key=lambda x: (x['page_number'], x['y_coordinate']))

In [5]:
len(get_all_pdfs())

75

: 